# マッチング推定

## 概要

マッチング（matching）は、処置群と対照群を「共変量の分布が似ているペア」や「似たサンプル集合」に揃えて因果効果の推定を行う方法。

1. マッチングする：似ている処置群と対照群のペアを見つける
2. 似ているペアでestimandを推定する

### 仮定

共変量で交絡をコントロールするため、selection on observableを仮定する。

1. $(Y(0), Y(1)) \operatorname{\perp\!\!\perp} D \mid X$ (条件付き独立の仮定)
2. $0 < P(D=1 \mid X) < 1$（common support）


### マッチングの種類

| 方法                                       | 概要                                 | 特徴                 |
| ---------------------------------------- | ---------------------------------- | ------------------ |
| **Nearest Neighbor Matching (最近傍マッチング)** | 処置群の各サンプルに対して、共変量距離が最も近い対照群サンプルを選ぶ | 実装が簡単、外れ値や多対一対応に注意 |
| **Caliper Matching (キャリパーマッチング)**        | 一定の距離（キャリパー）以内のサンプルのみをマッチング        | 極端なペアを除外できる        |
| **Propensity Score Matching (PSM)**      | ロジスティック回帰などで推定した傾向スコアでマッチング        | 高次元共変量を1つのスコアに要約   |
| **Full Matching / Optimal Matching**     | 全データを使い、最適化的にマッチング                 | バイアスを減らしつつ全サンプルを活用 |
| **Coarsened Exact Matching (CEM)**       | 共変量を粗くビニングし、同じビン内だけで比較             | 直感的・バイアスコントロールが容易  |
| **Mahalanobis Matching**                 | 共分散を考慮した距離で最近傍マッチング                | 多変量距離を考慮           |


### マッチング後の推定

マッチング後の「似ているペア」を使い

- 平均の差の比較（単純平均差）
- 回帰モデル（マッチング後の共変量で補正）
- 重み付け（CEMやPSMから得られる重みを利用）

などを行う

### 推定するestimand

- ATT：処置群のサンプルを基準に、似ている対照群をマッチする（例：最近傍マッチング）
- ATE：処置群と対照群の重みを使って計算（傾向スコアで、$w = \frac{1}{p(X)}$（対照群）や$\frac{1}{1-p(X)}$（処置群）など）

| 記号                                                | 意味                                   |
| ------------------------------------------------- | ------------------------------------ |
| **ATE (Average Treatment Effect)**                | 全体母集団での平均処置効果：$E[Y(1)-Y(0)]$         |
| **ATT (Average Treatment effect on the Treated)** | 処置群に対する平均処置効果：$E[Y(1)-Y(0)\mid T=1]$ |
| **ATC (Average Treatment effect on the Control)** | 対照群に対する平均処置効果：$E[Y(1)-Y(0)\mid T=0]$ |


### 距離尺度

「似ているペア」の程度を測る方法

- 傾向スコア
- 完全一致
- 粗い完全一致
- ユークリッド距離やマハラノビス距離

### マッチングに使うデータの選定

- **最近傍マッチング（Nearest Neighbor Matching）**：距離が最も近いサンプルを選ぶ。シンプルだが傾向スコアの極端な値などに弱い
- **Full Matching / Optimal Matching**：全データを使い、最適化的にマッチング

## Exact Matching

共変量$X$が完全に一致するペア同士で比較する

### 傾向スコアマッチング

傾向スコアで近さを求める

:::{card} コード例（R）

[MatchIt: Getting Started](https://cran.r-project.org/web/packages/MatchIt/vignettes/MatchIt.html) のコードを見ると次のようになっている

```r
# コード例（R）
library("MatchIt")

# Full matching on a probit PS
m_out <- matchit(treat ~ age + educ + race + married + 
                 nodegree + re74 + re75,
                 data = lalonde,
                 method = "full",
                 distance = "glm",
                 link = "probit")
m_data <- match_data(m_out)

library("marginaleffects")
fit <- lm(re78 ~ treat * (age + educ + race + married +
                            nodegree + re74 + re75),
          data = m.data,
          weights = weights)

avg_comparisons(fit,
                variables = "treat",
                vcov = ~subclass,
                newdata = subset(treat == 1))
```
:::

## Coarsend Exact Matching

共変量をグルーピングする、つまり粒度を荒く（coarsen）したあと、Exact Matchingを行う



結構有名で、Rのパッケージもあり、アメリカの医薬品の規制当局からも科学的な定量評価法として認可を受けてるらしい（[CEM: Coarsened Exact Matching Software | GARY KING](https://gking.harvard.edu/cem)）

- 論文: [Iacus, S. M., King, G., & Porro, G. (2012). Causal inference without balance checking: Coarsened exact matching. Political analysis, 20(1), 1-24.](https://gking.harvard.edu/files/political_analysis-2011-iacus-pan_mpr013.pdf)



:::{card} matchitを使ったコード例（R）

```r
library(MatchIt)
library(estimatr)
m_out <- matchit(treat ~ age + agesq + agecube + educ + educsq + marr + nodegree +
                 black + hisp + re74 + re75 + u74 + u75 + interaction1,
                 data = nsw_dw_cpscontrol,
                 method = "cem",
                 distance = "logit")
m_data <- match.data(m_out)

# 重みを使って推定
m_ate <- lm_robust(re78 ~ treat,
                   data = m_data,
                   weights = m_data$weights)
```

[Causal Inference The Mixtape](https://mixtape.scunning.com/05-matching_and_subclassification) のサンプルコードより引用

:::